In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import seaborn as sns
import os
from glob import glob

# import ecCodes

In [54]:
# import cfgrib
# !pip install ecCodes
# !conda install eccodes
# !conda install -c conda-forge eccodes

In [55]:
# read table with params: 
outdata = '/Users/varyabazilova/Desktop/alluvial_fans/dem_all/all_points_so_far/10_datawrangl_formodel!!/out/'

dfclim = pd.read_csv(outdata + '2aug_morph_temperature.csv')

In [4]:
dfclim.head()

,Unnamed: 0,DN,target,Name,x_centroid,y_centroid,area_m,perimeter,glaciersum,mean_elevation,...,Rc,Cc,mean_annual_T,mean_jan_T,mean_july_T,mean_monsoon_T,mean_no_monsoon_T,count_zeros,belowzero_frac,avgtemp_belowzero
0,0,2,1,DF,6533932.497,-500205.440,323908.568,3584.675,0.000,4738.964789,...,0.316762,1.789410,-4.698425,-13.451904,4.031158,0.353302,-9.77121,6.6875,0.66738,1
1,1,3,1,DF,6534113.836,-500799.045,182483.700,2922.889,0.000,4568.158333,...,0.268417,1.943889,-4.698425,-13.451904,4.031158,0.353302,-9.77121,6.6875,0.66738,1
2,3,4,1,DF,6533894.325,-503821.445,205294.163,2702.293,0.000,4182.414815,...,0.353282,1.694398,-4.698425,-13.451904,4.031158,0.353302,-9.77121,6.6875,0.66738,1
3,7,5,0,"Mixed - new looks FF, vegetated DF",6531952.768,-502250.739,8413968.867,19907.851,2763.123,4867.525384,...,0.266785,1.949823,-4.698425,-13.451904,4.031158,0.353302,-9.77121,6.6875,0.66738,1
4,8,7,0,FF - lower lef DF remnants?,6535837.722,-503989.910,359644.959,3750.121,0.000,4346.589852,...,0.321361,1.776559,-4.698425,-13.451904,4.031158,0.353302,-9.77121,6.6875,0.66738,1


1. Monsoon precipitation (sum of June-sept) 
2. Mean precipitation over year - daily
3. Annual sum 
4. Fraction of precipitation in monsoon 
5. P95% daily - sum in a day and then 95% 
6. Average number of wet days per year (frequency) 
7. Rain intensity on wet days 
8. Snow/no snow (fraction?) during wet days - if temp that day was below 0 precip_daymax_1990.nc
9. N-day sum (?) — rainy days (10 to 30) 
10. Monthly average -> cv (std/average) 

## just precipitations (no temps) 


### mean annual daily precipitation

In [26]:
# skip first element in path because its always ds_store
# paths = paths[1:]


In [57]:
%%time
# calculate mean annual temperature from daily data for each year: 

path = '/Users/varyabazilova/Desktop/era5land/dayly_precip'

meanannual = xr.Dataset()


for f in os.listdir(path)[1:]: # [1:] is to get rid of DS_store thing
    # print(f)
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # print(ds)
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # group by -> resample to annual 
    dsmean = ds.groupby('time.year').mean('time')
    # print(dsmean)
    # merge 
    meanannual = xr.merge([meanannual, dsmean])

# print(meanannual)

meanannual_mean = meanannual.mean(dim='year')

CPU times: user 18.9 s, sys: 16.7 s, total: 35.6 s
Wall time: 37 s


In [59]:
# select things 

# meanannual_mean

# select coordinates first 

lats = dfclim.y_wgs
lons = dfclim.x_wgs
meanannual_mean_sel = meanannual_mean.sel(latitude = lats, longitude = lons, method = 'nearest')

# overwrite the coordinates for them to match: (ovewrite dimentions)
meanannual_mean_sel['longitude'] = lons.values # x_wgs
meanannual_mean_sel['latitude']= lats.values   # y_wgs

In [60]:
%%time
# to dataframe 
meanannual_mean_sel_df = meanannual_mean_sel.to_dataframe()
# reset index 
meanannual_mean_sel_df = meanannual_mean_sel_df.reset_index()
# rename coordinates 
meanannual_mean_sel_df = meanannual_mean_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})
# m to mm 
meanannual_mean_sel_df['tp'] = meanannual_mean_sel_df.tp * 1000
# rename: 
meanannual_mean_sel_df = meanannual_mean_sel_df.rename(columns = {'tp':'mean_annual_tp'}) 


# merge:
dfclim = dfclim.merge(meanannual_mean_sel_df, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()



CPU times: user 121 ms, sys: 17.2 ms, total: 138 ms
Wall time: 150 ms


In [64]:
# dfclim.to_csv('out/3aug_dfclim.csv')

### mean annual sum 

In [65]:
%%time
# calculate mean annual temperature from daily data for each year: 

path = '/Users/varyabazilova/Desktop/era5land/dayly_precip'

sumannual = xr.Dataset()


for f in os.listdir(path)[1:]: # [1:] is to get rid of DS_store thing
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # group by -> resample to annual 
    dsmean = ds.groupby('time.year').sum('time')
    # print(dsmean)
    # merge 
    sumannual = xr.merge([sumannual, dsmean])

print(sumannual)



<xarray.Dataset>
Dimensions:    (year: 32, longitude: 691, latitude: 491)
Coordinates:
  * year       (year) int64 1990 1991 1992 1993 1994 ... 2018 2019 2020 2021
  * longitude  (longitude) float32 55.0 55.1 55.2 55.3 ... 123.8 123.9 124.0
  * latitude   (latitude) float32 55.0 54.9 54.8 54.7 54.6 ... 6.3 6.2 6.1 6.0
Data variables:
    tp         (year, latitude, longitude) float32 1.292 1.303 1.315 ... 0.0 0.0
CPU times: user 16.3 s, sys: 17 s, total: 33.3 s
Wall time: 35 s


In [66]:
# mean of the annual sum 
sumannual_mean = sumannual.mean(dim='year')

In [67]:
%%time

lats = dfclim.y_wgs
lons = dfclim.x_wgs

sumannual_mean_sel = sumannual_mean.sel(latitude = lats, longitude = lons, method = 'nearest')


# overwrite the coordinates for them to match: (ovewrite dimentions)
sumannual_mean_sel['longitude'] = lons.values # x_wgs
sumannual_mean_sel['latitude']= lats.values   # y_wgs

CPU times: user 12.7 ms, sys: 6.87 ms, total: 19.6 ms
Wall time: 19.7 ms


In [68]:
%%time
# to dataframe 
sumannual_mean_sel_df = sumannual_mean_sel.to_dataframe()
# reset index 
sumannual_mean_sel_df = sumannual_mean_sel_df.reset_index()
# rename coordinates 
sumannual_mean_sel_df = sumannual_mean_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})
# m to mm 
sumannual_mean_sel_df['tp'] = sumannual_mean_sel_df.tp * 1000
# rename: 
sumannual_mean_sel_df = sumannual_mean_sel_df.rename(columns = {'tp':'mean_annualsum_tp'}) 


# merge:
dfclim = dfclim.merge(sumannual_mean_sel_df, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()




CPU times: user 115 ms, sys: 30.2 ms, total: 145 ms
Wall time: 201 ms


In [70]:
# dfclim.to_csv('out/3aug_dfclim.csv')
# dfclim

## monsoon 

In [41]:
%%time
# calculate mean annual temperature from daily data for each year: 

path = '/Users/varyabazilova/Desktop/era5land/dayly_precip'

dsmonsoon = xr.Dataset()

for f in os.listdir(path)[1:]:       # [1:] is to get rid of DS_store thing
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # select monsoon month:
    dsmons = ds.sel(time=ds.time.dt.month.isin([4,5,6,7,8,9]))
    # merge 
    dsmonsoon = xr.merge([dsmonsoon, dsmons])
print('alltogether', dsmonsoon)



alltogether <xarray.Dataset>
Dimensions:    (time: 5856, longitude: 691, latitude: 491)
Coordinates:
  * time       (time) datetime64[ns] 1990-04-01T11:00:00 ... 2021-09-30T11:00:00
  * longitude  (longitude) float32 55.0 55.1 55.2 55.3 ... 123.8 123.9 124.0
  * latitude   (latitude) float32 55.0 54.9 54.8 54.7 54.6 ... 6.3 6.2 6.1 6.0
Data variables:
    tp         (time, latitude, longitude) float32 0.005021 0.004943 ... nan nan
CPU times: user 5min 28s, sys: 10min 43s, total: 16min 11s
Wall time: 22min 20s


## mean 

In [71]:
# calculate mean:
dsmonsoon_mean = dsmonsoon.mean(dim = 'time')


lats = dfclim.y_wgs
lons = dfclim.x_wgs

dsmonsoon_mean_sel = dsmonsoon_mean.sel(latitude = lats, longitude = lons, method = 'nearest')


# overwrite the coordinates for them to match: (ovewrite dimentions)
dsmonsoon_mean_sel['longitude'] = lons.values # x_wgs
dsmonsoon_mean_sel['latitude']= lats.values   # y_wgs

In [72]:
%%time

# to dataframe, clean up and merge 

# to dataframe 
dsmonsoon_mean_sel_df = dsmonsoon_mean_sel.to_dataframe()
# reset index 
dsmonsoon_mean_sel_df = dsmonsoon_mean_sel_df.reset_index()
# rename coordinates 
dsmonsoon_mean_sel_df = dsmonsoon_mean_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})
# m to mm 
dsmonsoon_mean_sel_df['tp'] = dsmonsoon_mean_sel_df.tp * 1000
# rename: 
dsmonsoon_mean_sel_df = dsmonsoon_mean_sel_df.rename(columns = {'tp':'mean_daylymonsoon_tp'}) 


# merge:
dfclim = dfclim.merge(dsmonsoon_mean_sel_df, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()





CPU times: user 114 ms, sys: 38.8 ms, total: 153 ms
Wall time: 204 ms


In [89]:

dfclim1 = dfclim.drop('mean_daylymonsoon_tp_x', axis = 1)
dfclim1 = dfclim1.rename(columns = {'mean_daylymonsoon_tp_y':'mean_daylymonsoon_tp'}) 

# dfclim1.to_csv('out/3aug_dfclim.csv')

In [90]:
dfclim1.columns    

Index(['Unnamed: 0', 'DN', 'target', 'Name', 'x_centroid', 'y_centroid',
       'area_m', 'perimeter', 'glaciersum', 'mean_elevation',
       'median_elevation', 'std_elevation', 'min_elevation', 'max_elevation',
       'range_elevation', 'variance_elevation', 'region', 'region_name',
       'mean_slope', 'median_slope', 'std_slope', 'min_slope', 'max_slope',
       'range_slope', 'variance_slope', 'glarea_percent', 'glacier', 'x_wgs',
       'y_wgs', 'M', 'Rc', 'Cc', 'mean_annual_T', 'mean_jan_T', 'mean_july_T',
       'mean_monsoon_T', 'mean_no_monsoon_T', 'count_zeros', 'belowzero_frac',
       'avgtemp_belowzero', 'mean_annual_tp', 'mean_annualsum_tp',
       'mean_daylymonsoon_tp', 'mean_monsoon_sum_tp', 'monsoon_tp_frac'],
      dtype='object')

### sum 

In [91]:
# calculate sum 
dsmonsoon_sum = dsmonsoon.sum(dim = 'time')

# select coordinates 
lats = dfclim.y_wgs
lons = dfclim.x_wgs

dsmonsoon_sum_sel = dsmonsoon_sum.sel(latitude = lats, longitude = lons, method = 'nearest')


# overwrite the coordinates for them to match: (ovewrite dimentions)
dsmonsoon_sum_sel['longitude'] = lons.values # x_wgs
dsmonsoon_sum_sel['latitude']= lats.values   # y_wgs


In [82]:
%%time

# to dataframe, clean up and merge 

# to dataframe 
# reset index 
dsmonsoon_sum_sel_df = dsmonsoon_sum_sel.to_dataframe().reset_index()
# rename coordinates 
dsmonsoon_sum_sel_df = dsmonsoon_sum_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs'})
# m to mm 
dsmonsoon_sum_sel_df['tp'] = dsmonsoon_sum_sel_df.tp * 1000
# rename: 
dsmonsoon_sum_sel_df = dsmonsoon_sum_sel_df.rename(columns = {'tp':'mean_monsoon_sum_tp'}) 


# merge:
dfclim = dfclim.merge(dsmonsoon_sum_sel_df, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()






CPU times: user 130 ms, sys: 35.9 ms, total: 165 ms
Wall time: 171 ms


In [80]:
# save dfclim 

# dfclim.columns
# dfclim.to_csv('out/3aug_dfclim.csv')



In [95]:
dfclim1.columns

Index(['Unnamed: 0', 'DN', 'target', 'Name', 'x_centroid', 'y_centroid',
       'area_m', 'perimeter', 'glaciersum', 'mean_elevation',
       'median_elevation', 'std_elevation', 'min_elevation', 'max_elevation',
       'range_elevation', 'variance_elevation', 'region', 'region_name',
       'mean_slope', 'median_slope', 'std_slope', 'min_slope', 'max_slope',
       'range_slope', 'variance_slope', 'glarea_percent', 'glacier', 'x_wgs',
       'y_wgs', 'M', 'Rc', 'Cc', 'mean_annual_T', 'mean_jan_T', 'mean_july_T',
       'mean_monsoon_T', 'mean_no_monsoon_T', 'count_zeros', 'belowzero_frac',
       'avgtemp_belowzero', 'mean_annual_tp', 'mean_annualsum_tp',
       'mean_daylymonsoon_tp', 'mean_monsoon_sum_tp', 'monsoon_tp_frac'],
      dtype='object')

## fraction of rain during monsoon 

In [97]:
# total monsoon / total annual precip 

dfclim1['monsoon_tp_frac'] = dfclim.mean_monsoon_sum_tp / dfclim.mean_annualsum_tp

In [99]:
dfclim1.to_csv('out/3aug_dfclim.csv')

### number of wet days (precip more them 10mm) a day 

In [115]:
%%time
# calculate how many times temp timeseries crosses 0 
path = '/Users/varyabazilova/Desktop/era5land/dayly_precip'

raindays = xr.Dataset()

for f in os.listdir(path)[1:]: 
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # m to mm 
    ds['tp'] = ds.tp * 1000
    # if negative = 1, else = 0 
    ds['forcount'] = xr.where(ds.tp > 5, 1, 0)
    # sum the "rainy days"
    dscount = ds.groupby('time.year').sum('time')
    # fraction of the year
    # dscount['fraction'] = dscount.forcount / 365
    
    # print('dsfraction', dscount)
    # merge:
    raindays = xr.merge([raindays, dscount])

print('number of days > 5 mm', raindays)

number of days <10mm <xarray.Dataset>
Dimensions:    (year: 32, longitude: 691, latitude: 491)
Coordinates:
  * year       (year) int64 1990 1991 1992 1993 1994 ... 2018 2019 2020 2021
  * longitude  (longitude) float32 55.0 55.1 55.2 55.3 ... 123.8 123.9 124.0
  * latitude   (latitude) float32 55.0 54.9 54.8 54.7 54.6 ... 6.3 6.2 6.1 6.0
Data variables:
    tp         (year, latitude, longitude) float32 1.292e+03 1.303e+03 ... 0.0
    forcount   (year, latitude, longitude) float64 90.0 92.0 99.0 ... 0.0 0.0
CPU times: user 36.2 s, sys: 49.9 s, total: 1min 26s
Wall time: 1min 34s


In [117]:
raindays_mean = raindays.mean(dim = 'year')

In [118]:
%%time


lats = dfclim.y_wgs
lons = dfclim.x_wgs

raindays_mean_sel = raindays_mean.sel(latitude = lats, longitude = lons, method = 'nearest')


# overwrite the coordinates for them to match: (ovewrite dimentions)
raindays_mean_sel['longitude'] = lons.values # x_wgs
raindays_mean_sel['latitude']= lats.values   # y_wgs

CPU times: user 16.3 ms, sys: 7.62 ms, total: 23.9 ms
Wall time: 22.6 ms


In [119]:
%%time

# to data frame
raindays_mean_sel_df = raindays_mean_sel.to_dataframe().reset_index().drop(['tp'], axis = 1)
# #rename:
raindays_mean_sel_df = raindays_mean_sel_df.rename(columns = {'longitude':'x_wgs', 'latitude':'y_wgs', 
                                                              'forcount': 'n_rainy_days'})
# #rename column: 
# # drop month 
# raindays_mean_sel_df = raindays_mean_sel_df

CPU times: user 32.5 ms, sys: 25.4 ms, total: 57.9 ms
Wall time: 75.4 ms


In [122]:
dfclim = dfclim.merge(raindays_mean_sel_df, on=['x_wgs', 'y_wgs'], how='left').drop_duplicates()


In [126]:
# dfclim.to_csv('out/dfclim_3aug_rainydays.csv')

### fraction of wet days (relative to 365)

In [128]:
# n_rainy_days/365

dfclim['rainy_days_frac'] = dfclim.n_rainy_days / 365

In [129]:
# dfclim.to_csv('out/dfclim_3aug_rainydays.csv')

### percentiles: 95% 


In [ ]:
%%time
# calculate how many times temp timeseries crosses 0 
path = '/Users/varyabazilova/Desktop/era5land/dayly_precip'

precentile95 = xr.Dataset()

for f in os.listdir(path)[1:]: 
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all")
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # m to mm 
    ds['tp'] = ds.tp * 1000
    # if negative = 1, else = 0 
    ds['forcount'] = xr.where(ds.tp > 5, 1, 0)
    # sum the "rainy days"
    dscount = ds.groupby('time.year').sum('time')
    # fraction of the year
    # dscount['fraction'] = dscount.forcount / 365
    
    # merge:
    precentile95 = xr.merge([precentile95, dscount])

print('precentile95', precentile95)

<xarray.Dataset>
Dimensions:   (y: 4)
Coordinates:
  * y         (y) float64 1.0 1.5 2.0 2.5
    quantile  float64 0.5
Data variables:
    a         (y) float64 3.6 5.75 6.0 1.7

In [140]:
da.quantile(0.5)#, dim="y")

<xarray.DataArray ()>
array(4.5)
Coordinates:
    quantile  float64 0.5